In [1]:
# Import necessary libraries
import pandas as pd
import requests

# Define the URLs for data sources
# BLS labor productivity data (tab-delimited file) and U.S. population data (JSON file)
BLS_PRODUCTIVITY_URL = "https://rearc-dataquest-s3bucket.s3.amazonaws.com/pr.data.0.Current"
US_POPULATION_URL = "https://rearc-dataquest-s3bucket.s3.amazonaws.com/nation_population.json"

# Load the Bureau of Labor Statistics productivity data from the S3 bucket
bls_productivity_df = pd.read_csv(BLS_PRODUCTIVITY_URL, delimiter="\t")

# Retrieve U.S. population data stored in JSON format from the S3 bucket
response = requests.get(US_POPULATION_URL)
population_json = response.json()

# Convert the nested JSON data into a flat tabular format (Pandas DataFrame)
us_population_df = pd.json_normalize(population_json, record_path="data")


In [2]:
# Filter Population Data for Year Range 2013 to 2018
# Convert the 'Year' column from string to integer
filtered_population_df = us_population_df[
    (us_population_df["Year"].astype(int) >= 2013) &
    (us_population_df["Year"].astype(int) <= 2018)
]
# Calculate 
# - Mean: Represents the average U.S. population over the 2013–2018 period
# - Standard Deviation: Indicates how much population values deviate from the mean
average_population = filtered_population_df["Population"].mean()
population_std_dev = filtered_population_df["Population"].std()

# Display the results 
print("U.S. Population Statistics (2013–2018):")
print(f"  • Average Population: {average_population:,.0f}")
print(f"  • Standard Deviation: {population_std_dev:,.0f}")

U.S. Population Statistics (2013–2018):
  • Average Population: 322,069,808
  • Standard Deviation: 4,158,441


In [3]:
# Display the current list of column names to identify any spacing issues
print(bls_productivity_df.columns.tolist())

['series_id        ', 'year', 'period', '       value', 'footnote_codes']


In [4]:
# Clean up whitespace around column names - series_id and value
bls_productivity_df.rename(columns={
    "series_id        ": "series_id",
    "       value": "value"
}, inplace=True)

In [5]:
# Generate Productivity Summary Report
# Step 1: Group the dataset by 'series_id' and 'year'
# Aggregate (sum) the 'value' field for each group to handle cases
# where multiple entries exist for the same series/year combination.
series_yearly_summary_df = (
    bls_productivity_df
    .groupby(["series_id", "year"], as_index=False)["value"]
    .agg("sum")
)

# Step 2: Sort the aggregated data by 'value' in descending order
# Then, for each 'series_id', keep only the record with the highest value.
# This effectively gives us the "maximum value year" for each productivity series.
max_value_per_series_df = (
    series_yearly_summary_df
    .sort_values("value", ascending=False)
    .drop_duplicates(subset="series_id", keep="first")
    .sort_index()
    .reset_index(drop=True)
)

# Display the Final Report
# The final DataFrame contains:
#   - series_id : Unique identifier for each productivity metric
#   - year      : Year in which the maximum value occurred
#   - value     : The corresponding maximum value
print("Maximum Productivity Value by Series:")
print(max_value_per_series_df)


Maximum Productivity Value by Series:
             series_id  year    value
0    PRS30006011        2022   20.500
1    PRS30006012        2022   17.100
2    PRS30006013        1998  705.895
3    PRS30006021        2010   17.700
4    PRS30006022        2010   12.400
..                 ...   ...      ...
277  PRS88003192        2002  282.800
278  PRS88003193        2024  860.838
279  PRS88003201        2022   37.200
280  PRS88003202        2022   28.700
281  PRS88003203        2024  583.441

[282 rows x 3 columns]


In [ ]:
# STEP 1: Filter Productivity Data for a Specific Series and Period
#   1. Filter rows matching the desired series_id pattern.
#   2. Further filter for the first-quarter records (Q01) only.

filtered_productivity_series_df = bls_productivity_df.loc[
    bls_productivity_df["series_id"].str.contains("PRS30006032", case=False)
]

# Narrow down the dataset to first-quarter (Q01) data
filtered_productivity_series_df = filtered_productivity_series_df[
    filtered_productivity_series_df["period"].str.contains("Q01", case=False)
]

print("Filtered Productivity Data (PRS30006032 - Q01):")
print(filtered_productivity_series_df.head())


# STEP 2: Prepare and Clean Population Data
#   Before merging with productivity data, ensure that the population dataset
#   has consistent numeric data types and column names suitable for merging.
#   1. Convert 'Population' to integer 
#   2. Convert 'Year' to integer 

us_population_df["Population"] = us_population_df["Population"].astype(int)
us_population_df["Year"] = us_population_df["Year"].astype(int)

print("\n Cleaned Population Data:")
print(us_population_df.head())


# STEP 3: Merge Productivity and Population Datasets
#   Combine both datasets to align productivity values with corresponding
#   U.S. population values for the same year.
#   - 'year' from productivity data matches 'Year' from population data
#   - Left join ensures all selected productivity records remain, even
#     if population data is missing for a particular year

merged_productivity_population_df = pd.merge(
    filtered_productivity_series_df,
    us_population_df,
    left_on="year",
    right_on="Year",
    how="left"
)

# STEP 4: The resulting dataset contains both productivity and population metrics.
#   - series_id:  Identifier for the productivity measure
#   - year:       Year of observation
#   - period:     Quarter (e.g., Q01)
#   - value:      Productivity measurement
#   - Population: U.S. population in that same year

final_combined_df = merged_productivity_population_df[
    ["series_id", "year", "period", "value", "Population"]
]

print("\n Combined Productivity and Population Data:")
print(final_combined_df)
